## Bahdanau et al. (2016)

https://arxiv.org/abs/1409.0473

https://machinelearningmastery.com/the-bahdanau-attention-mechanism/

Say we have a sequence (usually text) of n elements and it looks like this -

$$ X = x_1 , x_2 , ...... x_n $$

Which needs to be aligned with another sequence -

$$ Y = y_1 , y_2 , ...... y_n $$

And an encoder decoder network which in the paper is based on a bi-directional RNN. Traditional encoders create a fixed size vector from an input sequence, which limits the amount of information the decoder can use. Furthermore, the encoder has to create this vector from multi-dim hidden states, which may result in loss of information. 

What attention does is - 
- Finds the probability of an element in $X$, $x_j$ to be aligned with an element in the output sequence, $Y$, $y_i$
- The original work was for Machine Translation, hence the alignment modelling

So say the hidden state from the encoder is $h_j$ and from the decoder, $s_{i - 1}$

Then, 

attention probability for $y_i$ to $x_j$ is $\alpha_{ij} = softmax(e_{ij})$ (softmax since probability!)

now what is $e$ here? Well actually, the hidden state of the encoder contains a representations for all elements in $X$, and attention just doesn't look at one element on both sequences and finds a probability. It compares one element in the output against all the others in the input and says, how related is this element at $i$ in the output to $1....j$ elements in the input?

Why from output to input? Machine Translation, you're finding a proper translation for a source sentence. Or in other words, you want your decoder to search for the proper translation. Attention provides the search information for the decoder. So moving on, $e$, or the attention score comes from the attention module, which is defined and trained as a feed forward network.

$e_{ij} = a(s_{i -1}, h_j)$

And then, a context vector $c_i$ is created from $\alpha_{ij}$

$$
 c_i = \sum_{j = 1}^{n} \alpha_{ij}h_j
$$

In layman terms, find how probable $y_i$ is to be an output candidate based on the entire input $X$.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

So to be more specific, 

input goes to encoder -> gives hidden state $h_j$

$h_j$ goes to alignment module with the previous decoder state $s_{i - 1}$ -> gives attention score $e_{ij}$

take softmax of the attention score

get the context vector by weighted sum of annotation score and encoder hidden state


Let's generate some random tensors for the hidden states of encoder and the decoder.

In [2]:
# dummy input of size 50
x = torch.arange(50)

embedding = nn.Embedding(100, 20)
encoder = nn.RNN(20, 30, bidirectional=True)
decoder = nn.RNN(30, 30)

with torch.no_grad():
    x = embedding(x)
    _, h_j = encoder(x)


s_previous = h_j

There are two ways to get the attention score using the hidden states. 

1. Concatenate both and pass to a FFN with a single weight, $W$ (dot product)
2. Or, have two weights for each hidden state, apply the weights individually in a FFN and then add them (additive). 

in both cases the output of the FFN is passed to a tanh activation layer and then, a weight vector is applied for scaling.

The weight vectors are implemented as linear layers.

In [3]:
@torch.no_grad()
def attention(s_previous: torch.Tensor, h_j: torch.Tensor,  mode="dot") -> torch.Tensor:    
    if mode == "dot":
        # concat
        hidden_states = torch.cat([h_j, s_previous], dim=-1)
        
        # init weight
        # actually use a linear layer with no bias
        # output should be the size of h_j
        # well you have to multiply the attention score softmax with h_j!
        W = nn.Linear(hidden_states.size(-1), h_j.size(-1))
        
        # e_ij
        attention_scores = F.tanh(W(hidden_states))
        
        V = h_j
        attention_scores = V @ attention_scores.T
  
    elif mode == "add":
        W1 = nn.Linear(h_j.size(-1), h_j.size(-1))
        W2 = nn.Linear(s_previous.size(-1), s_previous.size(-1))
        
        attention_scores = F.tanh(W1(h_j) + W2(s_previous))
        
        V = h_j
        attention_scores = V @ attention_scores.T
        
    
    # transpose, since matmul for context vector
    alpha = F.softmax(attention_scores, dim=-1)
    c = alpha @ h_j
    
    return c

In [4]:
# concat
ctx = attention(s_previous, h_j)
print(ctx)
print(ctx.size())

tensor([[ 0.1693,  0.3710, -0.4432, -0.5377,  0.3817,  0.0567, -0.2332, -0.0859,
          0.0472, -0.0277, -0.0944,  0.4486, -0.0957, -0.1014,  0.4120,  0.1101,
         -0.4471, -0.2707, -0.2166,  0.0185,  0.6316, -0.0597,  0.0939,  0.0332,
         -0.0591,  0.0557,  0.2592, -0.3450,  0.6997, -0.3656],
        [ 0.4240,  0.0867, -0.4165, -0.6058,  0.4812,  0.2697, -0.0610,  0.0467,
         -0.2728,  0.3147,  0.0728,  0.1524,  0.1805, -0.4075,  0.6572,  0.0722,
         -0.5612, -0.3897, -0.2495,  0.1056,  0.4595, -0.0084,  0.4863, -0.2884,
         -0.1988,  0.2732, -0.0226, -0.3897,  0.6563, -0.3801]])
torch.Size([2, 30])


In [5]:
# additive
ctx = attention(s_previous, h_j, "add")
print(ctx)
print(ctx.size())

tensor([[ 0.0230,  0.5342, -0.4586, -0.4986,  0.3246, -0.0655, -0.3321, -0.1620,
          0.2309, -0.2242, -0.1904,  0.6186, -0.2543,  0.0743,  0.2712,  0.1319,
         -0.3816, -0.2024, -0.1977, -0.0315,  0.7304, -0.0892, -0.1313,  0.2178,
          0.0212, -0.0691,  0.4209, -0.3193,  0.7246, -0.3572],
        [ 0.2289,  0.3044, -0.4370, -0.5536,  0.4050,  0.1066, -0.1929, -0.0548,
         -0.0278,  0.0525, -0.0552,  0.3792, -0.0310, -0.1731,  0.4694,  0.1012,
         -0.4738, -0.2986, -0.2243,  0.0389,  0.5913, -0.0477,  0.1858, -0.0421,
         -0.0918,  0.1067,  0.1932, -0.3554,  0.6895, -0.3690]])
torch.Size([2, 30])
